In [112]:
import pandas as pd
import numpy as np
import csv
import statsmodels.api as sm
import warnings
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from itertools import combinations
from scipy import stats
from datetime import datetime
from sklearn.metrics import mean_absolute_error


file = 'rainfalldata.csv'
rd = pd.read_csv(file)
file2 = 'ncrainfalldata.csv'
ncrd = pd.read_csv(file2)
rd.Date = pd.to_datetime(rd.Date)
rd = rd.set_index('Date')
ncrd.Date = pd.to_datetime(ncrd.Date)
ncrd = ncrd.set_index('Date')

In [2]:
# this cell takes the stored exogen dictionary that is stored in the Data_Wrangling_CAP1 jupyter notebook
# that was imported above.
%store -r exogen


In [3]:
exogen.keys()

dict_keys(['Arcola, NC', 'Henderson 2 NNW, NC', 'Laurinburg, NC', 'Roanoke Rapids, NC', 'Murfreesboro, NC', 'Lumberton Area, NC', 'LONGWOOD, NC', 'WHITEVILLE 7 NW, NC', 'Charlotte Area, NC', 'Mount Mitchell Area, NC', 'ASHEVILLE AIRPORT, NC', 'BANNER ELK, NC', 'BEECH MOUNTAIN, NC', 'BRYSON CITY 4, NC', 'BREVARD, NC', 'CASAR, NC', 'COWEETA EXP STATION, NC', 'CULLOWHEE, NC', 'FOREST CITY 8 W, NC', 'FRANKLIN, NC', 'GASTONIA, NC', 'GRANDFATHER MTN, NC', ' HENDERSONVILLE 1 NE, NC', ' HIGHLANDS, NC', 'HOT SPRINGS, NC', 'LAKE LURE 2, NC', 'LAKE TOXAWAY 2 SW, NC', 'MARSHALL, NC', 'MONROE 2 SE, NC', ' MOUNT HOLLY 4 NE, NC', ' OCONALUFTEE, NC', 'PISGAH FOREST 3 NE, NC', 'ROBBINSVILLE AG 5 NE, NC', 'ROSMAN, NC', 'SHELBY 2 NW, NC', 'TAPOCO, NC', 'TRYON, NC', 'WAYNESVILLE 1 E, NC', 'Boone 1 SE, NC', 'DANBURY, NC', 'EDEN, NC', ' MOUNT AIRY 2 W, NC', 'REIDSVILLE 2 NW, NC', 'HAYESVILLE 1 NE, NC', 'MURPHY 4ESE, NC', ' KING, NC'])

In [4]:
def sarima_model_creation(data, p, d, q, P, D, Q, m, exog=None):
    my_order = [p,d,q]
    my_sorder = [P,D,Q,m]
    sarimamod = sm.tsa.statespace.SARIMAX(data, exog, order=my_order, seasonal_order=my_sorder, 
                                          enforce_stationarity=False, enforce_invertibility=False,
                                          initialization='approximate_diffuse')
    model_fit = sarimamod.fit()# start_params=[0, 0, 0, 0, 1])
    return(model_fit)

In [5]:
def hyperparameter_find(training_data, comb, testing_data, search = False, exogtr = None, exogtest = None):
    leastmae = 1000
    for com in tqdm(comb):
        li_one_step = []
        for i in tqdm(range(len(testing_data))):
            if i == 0:
                copytraining = training_data.copy()
                if exogtr is not None:
                    excopy = exogtr.copy()
                    mod_1 = sarima_model_creation(copytraining, com[0], 0, com[1], com[2], 0, 
                                                  com[3], 12, exog=excopy)
                    one_step_pred = mod_1.forecast(exog=excopy.iloc[[-1]]) #uses the data from the year before
                    excopy = pd.concat([excopy, exogtest.iloc[[i]]])
                else:
                    mod_1 = sarima_model_creation(copytraining, com[0], 0, com[1], com[2], 0, com[3], 12)
                    one_step_pred = mod_1.forecast()
                li_one_step.append(one_step_pred[0])
                copytraining = pd.concat([copytraining, testing_data[[i]]])
            else:
                if exogtr is not None:
                    mod_1 = sarima_model_creation(copytraining, com[0], 0, com[1], com[2], 0, 
                                                  com[3], 12, exog=excopy)
                    one_step_pred2 = mod_1.forecast(exog=excopy.iloc[[-1]])
                    excopy = pd.concat([excopy, exogtest.iloc[[i]]])
                else:
                    mod_1 = sarima_model_creation(copytraining, com[0], 0, com[1], com[2], 0, com[3], 12)
                    one_step_pred2 = mod_1.forecast()
                li_one_step.append(one_step_pred2[0])
                copytraining = pd.concat([copytraining, testing_data[[i]]])
        mae = mean_absolute_error(testing_data, li_one_step)
        if search is True:
            if mae < leastmae:
                leastmae = mae
                H_AR = com[0]
                H_MA = com[1]
                H_SAR = com[2]
                H_SMA = com[3]
            print(com,mae)            
    if search is True:
        return('AR: '+ str(H_AR), 'MA: ' +str(H_MA), 'SAR: '+str(H_SAR), 'SMA: '+str(H_SMA))
    else:
        return(mae)

In [6]:
def exog_combinations(df, exoe):
    lo_dfs = []
    if len(exoe) == 1:
        lo_dfs.append(df.loc[:,exoe])
    if len(exoe) > 1:
        lo_dfs.append(df.loc[:,exoe])
        for ex in exoe:
            lo_dfs.append(df.loc[:,[ex]])
        if len(exoe) >2:
            for i in range(2, len(exoe)):
                combolist = list(combinations(exoe,i))
                for c in combolist:
                    lo_dfs.append(df.loc[:,c])
    return(lo_dfs)


In [7]:
todokeys = ('TAPOCO, NC', 'TRYON, NC', 'WAYNESVILLE 1 E, NC', 'Boone 1 SE, NC', 'DANBURY, NC', 'EDEN, NC', ' MOUNT AIRY 2 W, NC', 'REIDSVILLE 2 NW, NC', 'HAYESVILLE 1 NE, NC', 'MURPHY 4ESE, NC', ' KING, NC')
sub_exogen = {k: exogen[k] for k in todokeys}

In [8]:
from collections import defaultdict
l_o_dfs = defaultdict(list)
for key,value in tqdm(sub_exogen.items()):
    lo_dfs2 = exog_combinations(rd, value)
    l_o_dfs[key] = lo_dfs2
# l_o_dfs['ROBBINSVILLE AG 5 NE, NC']

In [9]:
def exogenous_var(data, ncloc, l_exoloc, best_comb):
#     for key, value in tqdm(exo_dict.items()):
    dat = data[ncloc]
#         l_exog = exog_combinations(data, value)
    tr, test = train_test_split(dat, test_size = 0.2, shuffle=False)
    keymae = hyperparameter_find(tr, best_comb, test)
    print('keymae of: '+ key +' = '+str(keymae))
    bettermae = {}
    for exog in tqdm(l_exoloc):
        extr, extest = train_test_split(exog, test_size = 0.2, shuffle=False)
        exmae = hyperparameter_find(tr, best_comb, test, exogtr=extr, exogtest = extest)
        co = tuple(exog.columns)
        print('exmae = {}'.format(co) + ' '+ str(exmae))
        if exmae < keymae:
            bettermae[co] = exmae
            bettermae2 = {key: bettermae}
    return(co)

In [10]:
best_comb = [[4,3,3,4]]
warnings.filterwarnings("ignore")
for key,value in tqdm(l_o_dfs.items()):
    exogenous_var(rd, key, value, best_comb)

keymae of: TAPOCO, NC = 0.9800673566131274


exmae = ('GATLINBURG 2 SW, TN', 'NEWFOUND GAP, TN', ' TOWNSEND 5S, TN') 1.5764560295913925


exmae = ('GATLINBURG 2 SW, TN',) 1.4972163335360191


exmae = ('NEWFOUND GAP, TN',) 1.5501456737516661


exmae = (' TOWNSEND 5S, TN',) 2.1402191932997416


...

keymae of: WAYNESVILLE 1 E, NC = 1.747482139423062


exmae = ('MT LECONTE, TN', 'NEWFOUND GAP, TN') 2.2339993455520246


exmae = ('MT LECONTE, TN',) 2.067047382901454


exmae = ('NEWFOUND GAP, TN',) 2.262076438940648


MemoryError: 

In [16]:
warnings.filterwarnings("ignore")
new_model = sarima_model_creation(rd['Raleigh, NC'], 4,0,3,3,0,4,12)
pred = new_model.get_prediction(start='2019-05-01',end='2019-05-01')
pred

In [10]:
warnings.filterwarnings("ignore")
new_model1 = sarima_model_creation(ncrd['Raleigh, NC'], 4,0,3,3,0,4,12)
pred1 = new_model1.get_prediction(start='2019-05-01', end='2020-05-01')
pred1_ci = pred1.conf_int()


In [18]:
pred_df = pd.DataFrame(pred1_ci)
pred_df

,"lower Raleigh, NC","upper Raleigh, NC"
2019-05-01,-0.947526,7.680326
2019-06-01,-0.004114,8.626520
2019-07-01,1.995099,10.634899
2019-08-01,-0.191174,8.482544
2019-09-01,0.538479,9.212324
2019-10-01,0.275670,8.965745
2019-11-01,-1.515818,7.188782
2019-12-01,-1.325686,7.382033
2020-01-01,-0.365717,8.358265
2020-02-01,-1.540801,7.186353


In [48]:
new_model2 = sarima_model_creation(rd['TAPOCO, NC'], 4,0,3,3,0,4,12)
pred2 = new_model1.get_prediction(start='2020-06-01', end='2021-06-01')
pred3 = new_model2.get_prediction(start='2020-06-01', end='2021-06-01')
pred2_ci = pred2.conf_int()
pred3_ci = pred3.conf_int()
#pred_df[['upper'+ 'TAPOCO, NC'],['lower','TAPOCO, NC']] = pred3_ci[0]
pred2_ci = pd.merge(pred2_ci,pred3_ci, left_index=True, right_index=True)
pred2_ci

,"lower Raleigh, NC","upper Raleigh, NC","lower TAPOCO, NC","upper TAPOCO, NC"
2020-06-01,-0.251282,8.506845,0.913693,9.725917
2020-07-01,0.670969,9.435524,0.791939,9.617042
2020-08-01,0.871720,9.636611,0.544225,9.383828
2020-09-01,-0.052724,8.720575,0.534971,9.383312
2020-10-01,-0.778232,7.997310,0.370574,9.229937
2020-11-01,-0.982787,7.794767,0.211632,9.079761
2020-12-01,-1.214331,7.569436,0.300469,9.178035
2021-01-01,-0.939077,7.844943,0.648175,9.534577
2021-02-01,-1.154239,7.632554,0.602742,9.498070
2021-03-01,-0.018387,8.771740,0.835467,9.739591


In [110]:
with_exogs = ['WHITEVILLE 7 NW, NC', 'CASAR, NC', 'FOREST CITY 8 W, NC', 'GASTONIA, NC', 'LAKE LURE 2, NC', 
                       'ELIZABETHTOWN, NC', ' MOUNT HOLLY 4 NE, NC','GRANDFATHER MTN, NC']
ncrd2 = ncrd.copy()
ncrd_less = ncrd2.drop(with_exogs,axis=1)

In [64]:
len(ncrd_less.columns)

104

In [72]:
def prediction_fx(data, begin, end):
    prediction1_df = pd.DataFrame()
    one_step_pred1_df = pd.DataFrame()
    for idx, col in enumerate(data.columns):
        loc = data[col]
        mod_fit1 = sarima_model_creation(loc, 4,0,3,3,0,4,12)
        point_predictions = pd.DataFrame(mod_fit1.predict(start=begin, end=end))
        future_pred1 = mod_fit1.get_prediction(start=begin, end=end)
        future_pred1_ci = future_pred1.conf_int()
        point_predictions_df = pd.merge(point_predictions, future_pred1_ci, left_index=True, right_index=True)
        if idx is not 0:
            prediction1_df = pd.merge(prediction1_df, point_predictions_df, left_index=True, right_index=True)
        else:
            prediction1_df = prediction1_df.append(point_predictions_df)
    return(prediction1_df)

In [65]:
pre_df = prediction_fx(ncrd_less, '2019-05-01', '2069-05-01')
pre_df.head()

            Raleigh, NC  Fayetteville, NC  Albemarle, NC  Arcola, NC  \
2019-05-01       3.3664          3.099435       3.468173    4.087902   

            Asheboro, NC  Burlington, NC  Carthage, NC  Chapel Hill, NC  \
2019-05-01      3.503088        4.297819      2.953311         3.713626   

            Clayton, NC  Dunn, NC  ...  JEFFERSON 2 E, NC  \
2019-05-01     5.155408  4.249227  ...           4.932211   

             MOUNT AIRY 2 W, NC  NORTH WILKESBORO, NC  REIDSVILLE 2 NW, NC  \
2019-05-01             4.837811              4.686574             4.200847   

            TRANSOU, NC  W. KERR SCOTT RESV., NC  YADKINVILLE 6 E, NC  \
2019-05-01     6.836631                 4.764029             4.519221   

            HAYESVILLE 1 NE, NC  MURPHY 4ESE, NC   KING, NC  
2019-05-01              6.08703         6.056021   4.547799  

[1 rows x 104 columns]


,"lower Raleigh, NC","upper Raleigh, NC","lower Fayetteville, NC","upper Fayetteville, NC","lower Albemarle, NC","upper Albemarle, NC","lower Arcola, NC","upper Arcola, NC","lower Asheboro, NC","upper Asheboro, NC",...,"lower W. KERR SCOTT RESV., NC","upper W. KERR SCOTT RESV., NC","lower YADKINVILLE 6 E, NC","upper YADKINVILLE 6 E, NC","lower HAYESVILLE 1 NE, NC","upper HAYESVILLE 1 NE, NC","lower MURPHY 4ESE, NC","upper MURPHY 4ESE, NC","lower KING, NC","upper KING, NC"
2019-06-01,-0.004114,8.626520,-0.012104,9.297327,0.306195,9.139067,-0.178581,9.001982,-0.504364,8.306271,...,0.500192,9.450273,0.270625,8.011189,1.696870,9.732161,1.931865,10.420410,-0.348204,7.328047
2019-07-01,1.995099,10.634899,0.065915,9.376691,0.731770,9.626414,-0.545996,8.653015,0.784409,9.628806,...,0.300851,9.252151,1.010835,8.758913,2.033271,10.143700,1.746337,10.262593,0.208583,7.900919
2019-08-01,-0.191174,8.482544,0.522620,9.838840,0.044665,8.979854,-0.684326,8.514667,-0.928622,7.920682,...,0.410740,9.362254,0.969339,8.727245,1.018564,9.225303,0.566264,9.135823,0.454483,8.155193
2019-09-01,0.538479,9.212324,-0.045745,9.280859,-0.974847,7.961666,-0.034247,9.164743,0.066303,8.915642,...,0.840538,9.796385,0.236135,8.001192,0.961616,9.184278,0.405384,9.038031,0.581206,8.284420
2019-10-01,0.275670,8.965745,-1.202405,8.125620,-0.816096,8.124497,-0.363997,8.836460,-0.350810,8.517768,...,0.700389,9.657202,-0.327983,7.437745,0.139229,8.380348,-0.436586,8.223269,0.366808,8.071606
2019-11-01,-1.515818,7.188782,-1.830448,7.499798,-0.899592,8.042397,-0.473276,8.727931,-0.955412,7.915909,...,0.498071,9.455014,-0.649488,7.123658,1.006606,9.249372,0.115217,8.788236,0.416606,8.121630
2019-12-01,-1.325686,7.382033,-1.610475,7.720929,-0.531752,8.410329,-0.497145,8.704579,-0.944056,7.929537,...,0.836760,9.797460,0.020429,7.793604,1.722682,9.968097,0.952758,9.628444,0.663487,8.368785
2020-01-01,-0.365717,8.358265,-1.772823,7.559157,-1.471286,7.471375,-0.734122,8.467893,-0.722442,8.163052,...,0.316653,9.278740,-0.341158,7.435730,1.347452,9.592935,0.652386,9.329392,0.275711,7.980985
2020-02-01,-1.540801,7.186353,-1.901456,7.430935,-1.263183,7.678979,-0.769094,8.433354,-1.162874,7.722934,...,0.233612,9.195809,-0.828491,6.948020,1.499970,9.745691,1.104871,9.781685,0.167650,7.872956
2020-03-01,-0.619284,8.114632,-1.438091,7.894476,-0.279225,8.662980,-0.784691,8.418835,-1.062280,7.828564,...,0.143123,9.108491,-0.279739,7.496804,1.718835,9.964274,0.722183,9.398986,-0.217540,7.487770


In [66]:
rest_one_df, rest_pre_df = prediction_fx(ncrd_less, '2020-07-01', '2069-05-01')
rest_pre_df

,"lower Raleigh, NC","upper Raleigh, NC","lower Fayetteville, NC","upper Fayetteville, NC","lower Albemarle, NC","upper Albemarle, NC","lower Arcola, NC","upper Arcola, NC","lower Asheboro, NC","upper Asheboro, NC",...,"lower W. KERR SCOTT RESV., NC","upper W. KERR SCOTT RESV., NC","lower YADKINVILLE 6 E, NC","upper YADKINVILLE 6 E, NC","lower HAYESVILLE 1 NE, NC","upper HAYESVILLE 1 NE, NC","lower MURPHY 4ESE, NC","upper MURPHY 4ESE, NC","lower KING, NC","upper KING, NC"
2020-07-01,0.670969,9.435524,0.192565,9.527415,0.610259,9.556711,-0.371057,8.891575,-0.465321,8.441845,...,0.489473,9.466364,0.552304,8.350530,1.571594,9.838697,1.293592,9.973861,0.133756,7.866995
2020-08-01,0.871720,9.636611,0.467233,9.802085,-0.278216,8.668085,-0.492041,8.770584,-0.490761,8.417526,...,0.394381,9.371428,0.710478,8.509929,0.855963,9.123688,0.957393,9.637697,-0.049704,7.683603
2020-09-01,-0.052724,8.720575,-0.202586,9.132296,-0.328980,8.617230,-0.113186,9.149957,0.302270,9.215080,...,0.324734,9.304054,0.128112,7.927566,0.938361,9.206049,0.314288,8.994569,-0.181033,7.552261
2020-10-01,-0.778232,7.997310,-1.406804,7.928082,-2.193792,6.752422,-0.065320,9.197850,-1.233541,7.679749,...,0.306447,9.288335,-0.544713,7.254731,0.373805,8.641618,-0.736728,7.943565,-0.293157,7.440144
2020-11-01,-0.982787,7.794767,-1.966309,7.368583,-1.930717,7.015469,-0.847857,8.415487,-0.880312,8.035099,...,0.097302,9.079405,-0.892698,6.907973,1.180722,9.448522,1.123648,9.803935,-0.367039,7.366273
2020-12-01,-1.214331,7.569436,-1.712054,7.622842,-1.891985,7.054164,-0.805516,8.457975,-1.034422,7.883755,...,0.342468,9.326395,-0.296213,7.505504,1.977777,10.245595,2.192002,10.872280,-0.286830,7.446474
2021-01-01,-0.939077,7.844943,-1.725463,7.609434,-0.744751,8.201391,-0.631253,8.632226,-1.160133,7.758048,...,0.412278,9.399008,-0.590684,7.212674,1.409783,9.677598,0.998965,9.679235,-0.041375,7.691951
2021-02-01,-1.154239,7.632554,-1.979531,7.355358,-1.634477,7.311132,-1.099394,8.164443,-1.204407,7.716159,...,0.093409,9.080446,-0.826369,6.977284,1.588700,9.856402,1.478628,10.158584,-0.259062,7.474252
2021-03-01,-0.018387,8.771740,-1.419184,7.915657,-0.424991,8.520595,-0.832476,8.431335,-0.536403,8.385311,...,0.253159,9.241653,-0.264914,7.538793,1.483123,9.750512,0.666117,9.345897,0.088634,7.821972
2021-04-01,-0.582846,8.207287,-1.672177,7.662386,-1.270338,7.674950,-0.113607,9.150151,-0.693151,8.228903,...,0.691826,9.683254,0.088766,7.892505,0.867866,9.135238,0.263887,8.943536,0.572620,8.305947


In [38]:
with_exogs.pop(5)
exogenous_var_select = {k: exogen[k] for k in with_exogs}
exogenous_var_select

{'WHITEVILLE 7 NW, NC': [' LORIS 2 S, SC'],
 'CASAR, NC': ['CHESNEE 7 WSW, SC', 'GAFFNEY 6 E, SC'],
 'FOREST CITY 8 W, NC': ['CHESNEE 7 WSW, SC',
  'GAFFNEY 6 E, SC',
  'SPARTANBURG 3 SSE, SC'],
 'GASTONIA, NC': ['FORT MILL 4 NW, SC', 'GAFFNEY 6 E, SC'],
 'LAKE LURE 2, NC': ['CHESNEE 7 WSW, SC', 'CLEVELAND 3S, SC'],
 ' MOUNT HOLLY 4 NE, NC': ['FORT MILL 4 NW, SC'],
 'GRANDFATHER MTN, NC': ['ELIZABETHTON, TN', 'ROAN MOUNTAIN 3SW, TN']}

In [176]:
for key,value in exo_var_dict.items():
    for v in value:
        mod_fitttt = sarima_model_creation(v,4,0,3,3,0,4, 12)
        print(v.head())

             LORIS 2 S, SC
Date                      
1980-01-01            4.22
1980-02-01            2.10
1980-03-01            8.24
1980-04-01            1.40
1980-05-01            4.52


ValueError: Invalid value for design matrix. Requires a 2- or 3-dimensional array, got 1 dimensions

In [193]:
ches_gaff = rd[['CHESNEE 7 WSW, SC', 'GAFFNEY 6 E, SC']]
# moooo = sarima_model_creation(ches_gaff, 4,0,3,3,0,4, 12)
ches_gaff.iloc[:,0]

Date
1980-01-01     4.75
1980-02-01     1.02
1980-03-01    10.56
1980-04-01     4.16
1980-05-01    10.08
1980-06-01     5.39
1980-07-01     3.45
1980-08-01     4.41
1980-09-01     7.32
1980-10-01     4.12
1980-11-01     4.38
1980-12-01     0.68
1981-01-01     0.14
1981-02-01     3.37
1981-03-01     3.39
1981-04-01     1.03
1981-05-01     5.75
1981-06-01     5.63
1981-07-01     3.47
1981-08-01     1.67
1981-09-01     2.26
1981-10-01     4.52
1981-11-01     1.13
1981-12-01     7.60
1982-01-01     6.67
1982-02-01     6.38
1982-03-01     1.65
1982-04-01     4.45
1982-05-01     3.40
1982-06-01     6.44
              ...  
2016-11-01     1.27
2016-12-01     2.54
2017-01-01     5.65
2017-02-01     1.33
2017-03-01     2.98
2017-04-01     6.96
2017-05-01     5.33
2017-06-01     5.51
2017-07-01     6.21
2017-08-01     4.42
2017-09-01     4.25
2017-10-01     5.23
2017-11-01     0.84
2017-12-01     2.88
2018-01-01     2.82
2018-02-01     5.69
2018-03-01     3.94
2018-04-01     6.64
2018-05-01     

In [128]:
exo_var_dict = {
    'WHITEVILLE 7 NW, NC': [rd[[' LORIS 2 S, SC']]],
    'CASAR, NC': [rd[['CHESNEE 7 WSW, SC', 'GAFFNEY 6 E, SC']],
                rd[['CHESNEE 7 WSW, SC']],
                rd[['GAFFNEY 6 E, SC']]],
    'FOREST CITY 8 W, NC': [rd[['GAFFNEY 6 E, SC']], rd[['GAFFNEY 6 E, SC','SPARTANBURG 3 SSE, SC']]],
    'GASTONIA, NC': [rd[['FORT MILL 4 NW, SC','GAFFNEY 6 E, SC']], rd[['GAFFNEY 6 E, SC']]],
    'LAKE LURE 2, NC': [rd[['CHESNEE 7 WSW, SC']]],
    ' MOUNT HOLLY 4 NE, NC': [rd[['CHESNEE 7 WSW, SC']],rd[['CHESTER 1 SE, SC']],rd[['GAFFNEY 6 E, SC']], 
                              rd[['LOCKHART, SC']],
                             rd[['CATAWBA, SC','GAFFNEY 6 E, SC']],
                             rd[['CHESNEE 7 WSW, SC','GAFFNEY 6 E, SC']],
                             rd[['CHESNEE 7 WSW, SC', 'LOCKHART, SC']],
                             rd[['CHESTER 1 SE, SC', 'GAFFNEY 6 E, SC']],
                             rd[['CHESTER 1 SE, SC', 'LOCKHART, SC']],
                             rd[['FORT MILL 4 NW, SC', 'GAFFNEY 6 E, SC']],
                             rd[['GAFFNEY 6 E, SC', 'LOCKHART, SC']],
                             rd[['CATAWBA, SC', 'CHESNEE 7 WSW, SC', 'CHESTER 1 SE, SC']],
                             rd[['CATAWBA, SC', 'CHESNEE 7 WSW, SC', 'GAFFNEY 6 E, SC']],
                             rd[['CATAWBA, SC', 'CHESNEE 7 WSW, SC', 'LOCKHART, SC']],
                             rd[['CATAWBA, SC', 'CHESTER 1 SE, SC', 'LOCKHART, SC']],
                             rd[['CATAWBA, SC', 'CHESTER 1 SE, SC', 'GAFFNEY 6 E, SC']]]
}

In [129]:
exo_var_dict_12 = {
    'WHITEVILLE 7 NW, NC': [rd[' LORIS 2 S, SC']],
    'CASAR, NC': [rd['GAFFNEY 6 E, SC']],
    'FOREST CITY 8 W, NC': [rd['GAFFNEY 6 E, SC']],
    'GASTONIA, NC': [rd['GAFFNEY 6 E, SC']],
    'GRANDFATHER MTN, NC': [rd['ELIZABETHTON, TN']]
}

In [139]:
keys_list = list(exo_var_dict.keys())
locs_w_exogs = rd[keys_list]
keys_list12 = list(exo_var_dict_12.keys())
locs_w_exogs12 = rd[keys_list12]

In [164]:
exog_preds_df = pd.DataFrame(exog_preds)
exog_preds_df

,0
2019-05-01,2.786354
2019-06-01,2.852567
2019-07-01,2.994369
2019-08-01,2.936023
2019-09-01,2.564894
2019-10-01,2.103971
2019-11-01,2.177715
2019-12-01,2.012871


In [165]:
model_test = sarima_model_creation(white,4,0,3,3,0,4, 12,exog=rd[[' LORIS 2 S, SC']])
exog_model = sarima_model_creation(rd[[' LORIS 2 S, SC']],4,0,3,3,0,4, 12)
exog_preds = exog_model.forecast(steps=8)
exog_preds_df = pd.DataFrame(exog_preds)
model_test.predict(exog= exog_preds_df, steps=8)

2019-05-01    5.752936
2019-06-01    3.727615
2019-07-01    5.556847
2019-08-01    6.958229
2019-09-01    5.757391
2019-10-01    3.488874
2019-11-01    3.840373
2019-12-01    3.642856
Freq: MS, dtype: float64

In [167]:
preds = model_test.get_prediction(exog= exog_preds_df, start='2019-05-01', end='2019-12-01')
preds_ci = preds.conf_int()
preds_ci

,"lower WHITEVILLE 7 NW, NC","upper WHITEVILLE 7 NW, NC"
2019-05-01,1.437903,10.067969
2019-06-01,-0.623022,8.078252
2019-07-01,1.196296,9.917398
2019-08-01,2.586326,11.330131
2019-09-01,1.383504,10.131279
2019-10-01,-0.894071,7.871818
2019-11-01,-0.550817,8.231563
2019-12-01,-0.748330,8.034042


In [201]:
def prediction_exog_fx(data, exog_dict, begin, end):
    prediction_df = pd.DataFrame()
    pred_val_df = pd.DataFrame()
    for idx, col in enumerate(data.columns):
        loc = data[col]
        for key,value in exog_dict.items():
            for v in value:
                mod_fit1 = sarima_model_creation(loc, 4,0,3,3,0,4, 12,exog=v)
                if v.shape[1] > 1:
                    shap = v.shape[1]
                    for i in range(shap):
                        exog_mod_fit = sarima_model_creation(v.iloc[:,i],4,0,3,3,0,4,12)
                        e_preds2 = pd.DataFrame(exog_mod_fit.predict(start=begin, end=end))
                        if i is 0:
                            exog_predictions_df = e_preds2.copy()
                        else:
                            exog_predictions_df = pd.merge(exog_predictions_df, e_preds2, left_index=True, 
                                                           right_index=True)
                else:
                    exog_mod_fit = sarima_model_creation(v, 4,0,3,3,0,4,12)
                    exog_predictions_df = pd.DataFrame(exog_mod_fit.predict(start=begin, end=end))
                future_pred = mod_fit1.get_prediction(exog=exog_predictions_df,start=begin, end=end)
                future_pred_ci = future_pred.conf_int()
                future_pred_val= pd.DataFrame(mod_fit1.predict(exog=exog_predictions_df, start=begin, end=end))
#                 future_pred_full = pd.merge(future_pred_val, future_pred_ci, left_index=True, right_index=True)
                if idx is not 0:
                    prediction_df = pd.merge(prediction_df, future_pred_ci, left_index=True, right_index=True)
                    pred_val_df = pd.merge(pred_val_df, future_pred_val, left_index=True, right_index=True)
                else:
                    prediction_df = prediction_df.append(future_pred_ci)
                    pred_val_df = pred_val_df.append(future_pred_val)
    return(pred_val_df, prediction_df)
        
        

In [202]:
values_df, ci_df = prediction_exog_fx(locs_w_exogs, exo_var_dict, '2019-05-01', '2069-05-01')

In [204]:
values_df.head()

,0_x,0_y,0_x,0_y,0_x,0_y,0_x,0_y,0_x,0_y,...,0_x,0_y,0_x,0_y,0_x,0_y,0_x,0_y,0_x,0_y
2019-05-01,5.752936,5.121703,3.943828,4.659487,3.550862,3.550862,4.183637,3.728654,3.550862,4.659487,...,3.825663,3.946508,3.716548,4.051264,2.9899,3.945896,3.740305,3.652649,3.545385,3.562207
2019-05-01,4.734135,5.121703,3.943828,4.659487,3.550862,3.550862,4.183637,3.728654,3.550862,4.659487,...,3.825663,3.946508,3.716548,4.051264,2.9899,3.945896,3.740305,3.652649,3.545385,3.562207
2019-05-01,4.716170,5.121703,3.943828,4.659487,3.550862,3.550862,4.183637,3.728654,3.550862,4.659487,...,3.825663,3.946508,3.716548,4.051264,2.9899,3.945896,3.740305,3.652649,3.545385,3.562207
2019-05-01,5.195482,5.121703,3.943828,4.659487,3.550862,3.550862,4.183637,3.728654,3.550862,4.659487,...,3.825663,3.946508,3.716548,4.051264,2.9899,3.945896,3.740305,3.652649,3.545385,3.562207
2019-05-01,5.195482,5.121703,3.943828,4.659487,3.550862,3.550862,4.183637,3.728654,3.550862,4.659487,...,3.825663,3.946508,3.716548,4.051264,2.9899,3.945896,3.740305,3.652649,3.545385,3.562207


In [ ]:
ci_merged = pd.merge(pre_df, ci_df, left_index=True, right_index=True)
all_merged = pd.merge(ci_merged, values_df, left_index=True, right_index=True)
all_merged.head()

In [ ]:
all_merged.to_csv('predictions.csv')